In [1]:
!pip install boto3 botocore


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import boto3
import botocore

# Retrieve AWS credentials and S3 connection details from environment variables
aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
endpoint_url = os.environ.get('AWS_S3_ENDPOINT')
region_name = os.environ.get('AWS_DEFAULT_REGION')
bucket_name = os.environ.get('AWS_S3_BUCKET')

# Check if all required environment variables are set
if not all([aws_access_key_id, aws_secret_access_key, endpoint_url, region_name, bucket_name]):
    raise ValueError("One or more data connection variables are empty. "
                     "Please check your data connection to an S3 bucket.")

# Create a session and S3 resource
session = boto3.session.Session(aws_access_key_id=aws_access_key_id,
                                aws_secret_access_key=aws_secret_access_key)

s3_resource = session.resource(
    's3',
    config=botocore.client.Config(signature_version='s3v4'),
    endpoint_url=endpoint_url,
    region_name=region_name)

bucket = s3_resource.Bucket(bucket_name)


def upload_file_to_s3(local_file_path, s3_key):
    """
    Uploads a single file to S3.
    
    Parameters:
    - local_file_path: Path to the local file to upload.
    - s3_key: Path within the S3 bucket to store the file.
    """
    if not os.path.isfile(local_file_path):
        raise ValueError(f"The file '{local_file_path}' does not exist.")
    
    print(f"Uploading {local_file_path} to s3://{bucket_name}/{s3_key}")
    bucket.upload_file(local_file_path, s3_key)
    print(f"File uploaded successfully to s3://{bucket_name}/{s3_key}")
    return 1  # Return 1 since only one file is uploaded


def list_objects(prefix):
    """
    Lists all objects in the S3 bucket with a given prefix.
    
    Parameters:
    - prefix: The prefix to filter objects in the S3 bucket.
    """
    filter = bucket.objects.filter(Prefix=prefix)
    for obj in filter.all():
        print(obj.key)


# Example usage of uploading specific files
# List of files to upload
files_to_upload = [
    "models/1/xgb_classifier.onnx",  # Specify each file path here
]

# Check if the files exist locally and upload them individually
for file_path in files_to_upload:
    # Extract the relative path within the bucket
    s3_key = file_path  # Adjust if you want a different path in S3
    upload_file_to_s3(file_path, s3_key)

# List the uploaded files in S3
print("\nUploaded files in S3 under 'models':")
list_objects("models")


Uploading models/1/xgb_classifier.onnx to s3://test/models/1/xgb_classifier.onnx
File uploaded successfully to s3://test/models/1/xgb_classifier.onnx

Uploaded files in S3 under 'models':
models/1/xgb_classifier.onnx
models/pytorch_model-00001-of-00002.bin
models/pytorch_model-00002-of-00002.bin
models/pytorch_model.bin.index.json
models/special_tokens_map.json
models/tokenizer.model
models/tokenizer_config.json
models/vicuna/README.md
models/vicuna/config.json
models/vicuna/generation_config.json
